In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist

# Load the MNIST data.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# If subtract pixel mean is enabled
x_train_mean = np.mean(x_train, axis=0)
x_train -= x_train_mean
x_test -= x_train_mean

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Input image dimensions.
input_shape = x_train.shape[1:]

Using TensorFlow backend.


In [2]:
from keras.layers import *
from model import LogGaussMF
from resnet_backend import *


# Prepare model model saving directory.
def get_callbacks(model_type):
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    model_name = "%s_model.{epoch:03d}.h5" % model_type
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    filepath = os.path.join(save_dir, model_name)

    # Prepare callbacks for model saving and for learning rate adjustment.
    checkpoint = ModelCheckpoint(
        filepath=filepath,
        monitor='val_acc',
        verbose=1,
        save_best_only=True)

    lr_scheduler = LearningRateScheduler(lr_schedule)

    lr_reducer = ReduceLROnPlateau(
        factor=np.sqrt(0.1),
        cooldown=0,
        patience=5,
        min_lr=0.5e-6)

    return [checkpoint, lr_reducer, lr_scheduler]


n = 2
depth = n * 9 + 2

# Model name, depth and version
model_type = 'Fuzzy-ResNet%dv%d' % (depth, 2)

inputs, features = resnet_backend_v2(
    input_shape=input_shape, 
    depth=depth)
memberships = LogGaussMF(10)(features)
rules = Lambda(lambda x: K.sum(x, axis=-1))(memberships)
linear = Dense(10)(features)
logits = Add()([rules, linear])
outputs = Activation("softmax")(logits)
model = Model(inputs=inputs, outputs=outputs)

model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(lr=lr_schedule(0)),
    metrics=['accuracy'])

callbacks = get_callbacks(model_type)

W0508 15:53:21.766034 140313594894144 deprecation.py:506] From /home/ryan-desktop/anaconda3/envs/keras/lib/python3.7/site-packages/tensorflow/python/training/moving_averages.py:210: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Learning rate:  0.001


In [3]:
fuzzy_history = model.fit(
    x=x_train, y=y_train,
    validation_data=(x_test, y_test),
    epochs=200,
    batch_size=32,
    verbose=1, 
    callbacks=callbacks)

Train on 60000 samples, validate on 10000 samples
Epoch 1/200
Learning rate:  0.001
60000/60000 [==============================] - 53s 878us/step - loss: 0.5098 - acc: 0.9465 - val_loss: 0.3994 - val_acc: 0.9530

Epoch 00001: val_acc improved from -inf to 0.95300, saving model to /home/ryan-desktop/Documents/research-2019/mnist/saved_models/Fuzzy-ResNet20v2_model.001.h5
Epoch 2/200
Learning rate:  0.001
60000/60000 [==============================] - 49s 809us/step - loss: 0.2762 - acc: 0.9784 - val_loss: 0.2319 - val_acc: 0.9834

Epoch 00002: val_acc improved from 0.95300 to 0.98340, saving model to /home/ryan-desktop/Documents/research-2019/mnist/saved_models/Fuzzy-ResNet20v2_model.002.h5
Epoch 3/200
Learning rate:  0.001
60000/60000 [==============================] - 49s 811us/step - loss: 0.2081 - acc: 0.9818 - val_loss: 0.2380 - val_acc: 0.9659

Epoch 00003: val_acc did not improve from 0.98340
Epoch 4/200
Learning rate:  0.001
60000/60000 [==============================] - 48s 805

60000/60000 [==============================] - 46s 765us/step - loss: 0.0502 - acc: 0.9939 - val_loss: 0.0737 - val_acc: 0.9887

Epoch 00036: val_acc did not improve from 0.99180
Epoch 37/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 766us/step - loss: 0.0511 - acc: 0.9938 - val_loss: 0.0620 - val_acc: 0.9910

Epoch 00037: val_acc did not improve from 0.99180
Epoch 38/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 770us/step - loss: 0.0506 - acc: 0.9935 - val_loss: 0.0772 - val_acc: 0.9865

Epoch 00038: val_acc did not improve from 0.99180
Epoch 39/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 767us/step - loss: 0.0487 - acc: 0.9943 - val_loss: 0.0762 - val_acc: 0.9872

Epoch 00039: val_acc did not improve from 0.99180
Epoch 40/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 767us/step - loss: 0.0490 - acc: 0.9941 - val_loss: 0.0718 - val_acc: 0.9877

Epoch 00040: va

60000/60000 [==============================] - 46s 765us/step - loss: 0.0415 - acc: 0.9946 - val_loss: 0.0678 - val_acc: 0.9860

Epoch 00074: val_acc did not improve from 0.99210
Epoch 75/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 766us/step - loss: 0.0416 - acc: 0.9947 - val_loss: 0.0633 - val_acc: 0.9897

Epoch 00075: val_acc did not improve from 0.99210
Epoch 76/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 766us/step - loss: 0.0391 - acc: 0.9954 - val_loss: 0.0671 - val_acc: 0.9865

Epoch 00076: val_acc did not improve from 0.99210
Epoch 77/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 768us/step - loss: 0.0396 - acc: 0.9951 - val_loss: 0.0674 - val_acc: 0.9874

Epoch 00077: val_acc did not improve from 0.99210
Epoch 78/200
Learning rate:  0.001
60000/60000 [==============================] - 46s 767us/step - loss: 0.0400 - acc: 0.9950 - val_loss: 0.0714 - val_acc: 0.9862

Epoch 00078: va


Epoch 00110: val_acc did not improve from 0.99430
Epoch 111/200
Learning rate:  0.0001
60000/60000 [==============================] - 46s 768us/step - loss: 0.0085 - acc: 0.9998 - val_loss: 0.0347 - val_acc: 0.9929

Epoch 00111: val_acc did not improve from 0.99430
Epoch 112/200
Learning rate:  0.0001
60000/60000 [==============================] - 46s 766us/step - loss: 0.0083 - acc: 0.9999 - val_loss: 0.0313 - val_acc: 0.9939

Epoch 00112: val_acc did not improve from 0.99430
Epoch 113/200
Learning rate:  0.0001
60000/60000 [==============================] - 46s 764us/step - loss: 0.0082 - acc: 0.9998 - val_loss: 0.0323 - val_acc: 0.9928

Epoch 00113: val_acc did not improve from 0.99430
Epoch 114/200
Learning rate:  0.0001
60000/60000 [==============================] - 46s 766us/step - loss: 0.0082 - acc: 0.9998 - val_loss: 0.0348 - val_acc: 0.9936

Epoch 00114: val_acc did not improve from 0.99430
Epoch 115/200
Learning rate:  0.0001
60000/60000 [==============================] - 4

60000/60000 [==============================] - 49s 822us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0299 - val_acc: 0.9943

Epoch 00185: val_acc did not improve from 0.99470
Epoch 186/200
Learning rate:  5e-07
60000/60000 [==============================] - 51s 855us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0301 - val_acc: 0.9941

Epoch 00186: val_acc did not improve from 0.99470
Epoch 187/200
Learning rate:  5e-07
60000/60000 [==============================] - 51s 854us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0302 - val_acc: 0.9941

Epoch 00187: val_acc did not improve from 0.99470
Epoch 188/200
Learning rate:  5e-07
60000/60000 [==============================] - 51s 851us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0301 - val_acc: 0.9942

Epoch 00188: val_acc did not improve from 0.99470
Epoch 189/200
Learning rate:  5e-07
60000/60000 [==============================] - 51s 850us/step - loss: 0.0045 - acc: 1.0000 - val_loss: 0.0303 - val_acc: 0.9940

Epoch 00189

In [4]:
# n = 2
# depth = n * 9 + 2

# # Model name, depth and version
# model_type = 'ResNet%dv%d' % (depth, 2)

# inputs, features = resnet_backend_v2(
#     input_shape=input_shape, 
#     depth=depth)
# memberships = LogGaussMF(10)(features)
# rules = Lambda(lambda x: K.sum(x, axis=-1))(memberships)
# linear = Dense(10)(features)
# logits = Add()([rules, linear])
# outputs = Activation("softmax")(logits)
# model = Model(inputs=inputs, outputs=outputs)

# model.compile(
#     loss='categorical_crossentropy',
#     optimizer=Adam(lr=lr_schedule(0)),
#     metrics=['accuracy'])

# callbacks = get_callbacks(model_type)

In [5]:
# history = model.fit(
#     x=x_train, y=y_train,
#     validation_data=(x_test, y_test),
#     epochs=200,
#     batch_size=32,
#     verbose=1, 
#     callbacks=callbacks)

In [6]:
# fig, axes = plt.subplots(5, 1, figsize=(12, 20), squeeze=True)
# axes[0].set_title("Loss")
# axes[0].plot(history.history["loss"], c="b")
# axes[0].plot(history.history["val_loss"], c="r")

# axes[1].set_title("Log Loss")
# axes[1].plot(np.log(history.history["loss"]), c="b")
# axes[1].plot(np.log(history.history["val_loss"]), c="r")

# axes[2].set_title("Accuracy")
# axes[2].plot(history.history["acc"], c="b")
# axes[2].plot(history.history["val_acc"], c="r")

# axes[3].set_title("Accuracy")
# axes[3].set_ylim((0.99, 1.01))
# axes[3].plot(history.history["acc"], c="b")
# axes[3].plot(history.history["val_acc"], c="r")

# axes[4].set_title("Log Learning Rate")
# axes[4].plot(np.log(history.history["lr"]), c="b")